In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
#!/home/ektov1-av_ca-sbrf-ru/bin/python35
import os
import sys
curruser = os.environ.get('USER')

sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

sys.path.insert(0, './src')

# import tendo.singleton
import warnings
warnings.filterwarnings('ignore')

# import logging
# logging.basicConfig(filename='./__upd_runGASiteProd2SAS__.log',level=logging.INFO,
#                     format='%(asctime)s %(levelname)s %(name)s %(message)s')
# logger = logging.getLogger(__name__)


import joblib
import json
from joblib import Parallel, delayed

from time import sleep
from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import subprocess
from threading import Thread
import time
from datetime import datetime

from spark_connector import SparkConnector
from sparkdb_loader import spark
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame

import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load
from etl import ETLORA


# sing = tendo.singleton.SingleInstance()

# os.chdir('/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/Clickstream_Analytics/AutoUpdate/')
# os.chdir('/opt/workspace/{}/notebooks/clickstream/AutoUpdate/'.format(curruser))


##-----------------------------------------
conn_schema = 'sbx_team_digitcamp'
ga_schema = 'sklod_external_google_analytics'
tbl = 'GA_SITE_ALL_PRODS'
out_table_name = 'GA_ALL_SCENARIOS_HIST'
##-----------------------------------------http://pklis-chd000224.labiac.df.sbrf.ru:8100/notebooks/ISKRA_GET_LOAD/Hive2Iskra.ipynb#

def show(self, n=10):
    return self.limit(n).toPandas()

pyspark.sql.dataframe.DataFrame.show = show

def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''

    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)

# @typed_udf(StringType())
def match_product(product_dict_nm):
    def get_product(product_norm, product_dict_nm):
        if product_norm is None:
            return None

        for product_nm in product_dict_nm:
            if product_nm in product_norm:
                return product_dict_nm[product_nm]
            else:
                return None

    return f.udf(lambda x: get_product(x,product_dict_nm))

In [3]:
print("### Starting spark context. Run!")

sp = spark(schema=conn_schema,
           dynamic_alloc=False,
           numofinstances=10, 
           numofcores=8,
           kerberos_auth=True,
           process_label="shaymanov-an"
           )
hive = sp.sql

print(sp.sc.version)

### Starting spark context. Run!
2.4.0.cloudera2


In [10]:
table_name = 'GA_ALL_SCENARIOS_HIST'
sdf = hive.sql('''select * from {}.{} 
                  where 1=1 and
                        inn = '260800070346' and
                        channel = 'SBBOL' and
                        id_product = 'DEPOSIT' and
                        (load_dt >= timestamp('2021-03-01') ) and 
                        (load_dt <= timestamp('2021-03-16') )'''.format(conn_schema,table_name))

In [11]:
sdf.show(40)

,inn,sbbolUserId,commonSegmentoUID,id_crm,scenario_id,channel,id_product,madeal_product_norm,madeal_prod_type_name,madeal_appl_core_txt,...,essense,returncnt,numberOfSteps,funnel_rate,sum_offer_priority,sum_offer_nontop,minSessionStartTime,maxSessionStartTime,load_dt,ctl_loading
0,260800070346,3c25f096-6d65-fc84-e054-90e2ba85d0c8,"JYJnvs19GKpU,uDsVktljQq3m,iuQ36w3RnyIG,UOFXBrl...",1-AWQLK8O,DEPOSIT_01,SBBOL,DEPOSIT,None,None,None,...,Scenario DEPOSIT_01 retargeting product DEPOSI...,1,3,0.33,0,0,2021-02-25 15:55:50,2021-02-25 15:55:50,2021-03-01,6749242


## Upload to ISKRA from Pandas df

In [6]:
df= sdf.toPandas()

In [119]:
df.dtypes

inn                              object
sbbolUserId                      object
commonSegmentoUID                object
id_crm                           object
scenario_id                      object
channel                          object
id_product                       object
madeal_product_norm              object
madeal_prod_type_name            object
madeal_appl_core_txt             object
madeal_complete_dt       datetime64[ns]
hasProduct                        int64
essense                          object
returncnt                         int64
numberOfSteps                     int64
funnel_rate                      object
sum_offer_priority                int64
sum_offer_nontop                  int64
minSessionStartTime      datetime64[ns]
maxSessionStartTime      datetime64[ns]
load_dt                  datetime64[ns]
ctl_loading                       int64
dtype: object

In [122]:
cols = [col for col in df.columns if col != "commonSegmentoUID"]+["commonSegmentoUID"]
df = df.loc[:,cols]

## Create empty table with predefined schema

In [135]:
str_ = 'CREATE TABLE {} ( '
for column_name, column in df.iteritems():
    if column.dtype.kind == 'O':
        if (df[column_name].str.len().max() >= 4000) or ('COMMONSEGMENTOUID' in column_name.upper()):
#             df.drop(columns=[column_name], inplace=True)
            str_+=column_name.upper() + ' ' +'CLOB, '
        else:
            if 'INN' in column_name.upper() or 'KPP' in column_name.upper():
                str_+=column_name.upper() + ' ' +'VARCHAR2(20), '
            else:
                str_+=column_name.upper() + ' ' +'VARCHAR2(800), '
    elif column.dtype.kind == 'i':
        str_+=column_name.upper() + ' ' +'NUMBER(20), '
    elif column.dtype.kind == 'f':
        df[column_name] = df[column_name].fillna(0.0)
        str_+=column_name.upper() + ' ' +'FLOAT(126), '        
    elif column.dtype.kind == 'M':
        str_+=column_name.upper() + ' ' +'TIMESTAMP, '
    else:
        None  
res=str_.strip()[:-1] + ' )'
res

'CREATE TABLE {} ( INN VARCHAR2(20), SBBOLUSERID VARCHAR2(800), ID_CRM VARCHAR2(800), SCENARIO_ID VARCHAR2(800), CHANNEL VARCHAR2(800), ID_PRODUCT VARCHAR2(800), MADEAL_PRODUCT_NORM VARCHAR2(800), MADEAL_PROD_TYPE_NAME VARCHAR2(800), MADEAL_APPL_CORE_TXT VARCHAR2(800), MADEAL_COMPLETE_DT TIMESTAMP, HASPRODUCT NUMBER(20), ESSENSE VARCHAR2(800), RETURNCNT NUMBER(20), NUMBEROFSTEPS NUMBER(20), FUNNEL_RATE VARCHAR2(800), SUM_OFFER_PRIORITY NUMBER(20), SUM_OFFER_NONTOP NUMBER(20), MINSESSIONSTARTTIME TIMESTAMP, MAXSESSIONSTARTTIME TIMESTAMP, LOAD_DT TIMESTAMP, CTL_LOADING NUMBER(20), COMMONSEGMENTOUID CLOB )'

In [3]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('tech_iskra[iskra]', 'Uthvfy123', ld._get_dsn('iskra4')) 
db.connect() 

In [4]:
sql = ''' select /*+ parallel (12) */ 
                 min(interaction_start_date),
                 max(interaction_start_date) 
          from AIST_INTERACTION_HISTORY '''
df2 = pd.read_sql(sql, con=db.connection)

In [5]:
df2.head()

,MIN(INTERACTION_START_DATE),MAX(INTERACTION_START_DATE)
0,2019-01-04 00:07:43,2021-06-24 20:57:10


In [137]:
table_name = 'tmp_GA_ALL_SCENARIOS_HIST'

In [138]:
sql = "DROP TABLE {}".format(table_name)
db.cursor.execute(sql)
db.connection.commit()
print('dropping done')

dropping done


In [139]:
sql = res.format(table_name)

db.cursor.execute(sql)
db.connection.commit()
print('creating done')

creating done


In [140]:
# table_name = 'tmp_GA_ALL_SCENARIOS_HIST'
ld.upload_df_or_csv(df, table_name, parallel=1, 
                    password='Uthvfy123', path= None,
                    isclobe=1, isuseclobdct=1, verbose=1, njobs=5)

OrderedDict([('INN', <cx_Oracle.DbType DB_TYPE_LONG>), ('SBBOLUSERID', <cx_Oracle.DbType DB_TYPE_LONG>), ('ID_CRM', <cx_Oracle.DbType DB_TYPE_LONG>), ('SCENARIO_ID', <cx_Oracle.DbType DB_TYPE_LONG>), ('CHANNEL', <cx_Oracle.DbType DB_TYPE_LONG>), ('ID_PRODUCT', <cx_Oracle.DbType DB_TYPE_LONG>), ('MADEAL_PRODUCT_NORM', <cx_Oracle.DbType DB_TYPE_LONG>), ('MADEAL_PROD_TYPE_NAME', <cx_Oracle.DbType DB_TYPE_LONG>), ('MADEAL_APPL_CORE_TXT', <cx_Oracle.DbType DB_TYPE_LONG>), ('MADEAL_COMPLETE_DT', <cx_Oracle.DbType DB_TYPE_TIMESTAMP>), ('HASPRODUCT', <cx_Oracle.ApiType NUMBER>), ('ESSENSE', <cx_Oracle.DbType DB_TYPE_LONG>), ('RETURNCNT', <cx_Oracle.ApiType NUMBER>), ('NUMBEROFSTEPS', <cx_Oracle.ApiType NUMBER>), ('FUNNEL_RATE', <cx_Oracle.DbType DB_TYPE_LONG>), ('SUM_OFFER_PRIORITY', <cx_Oracle.ApiType NUMBER>), ('SUM_OFFER_NONTOP', <cx_Oracle.ApiType NUMBER>), ('MINSESSIONSTARTTIME', <cx_Oracle.DbType DB_TYPE_TIMESTAMP>), ('MAXSESSIONSTARTTIME', <cx_Oracle.DbType DB_TYPE_TIMESTAMP>), ('LOAD_D

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   6 out of   6 | elapsed:  1.3min finished


## Check saved DF via cx_Oracle

In [141]:
query = '''
select /*+ parallel (12) */
          *
FROM {}
where rownum < 1000
'''.format(table_name)

In [142]:
df2 = pd.read_sql(query, con=db.connection)

In [143]:
df2.head()

,INN,SBBOLUSERID,ID_CRM,SCENARIO_ID,CHANNEL,ID_PRODUCT,MADEAL_PRODUCT_NORM,MADEAL_PROD_TYPE_NAME,MADEAL_APPL_CORE_TXT,MADEAL_COMPLETE_DT,...,RETURNCNT,NUMBEROFSTEPS,FUNNEL_RATE,SUM_OFFER_PRIORITY,SUM_OFFER_NONTOP,MINSESSIONSTARTTIME,MAXSESSIONSTARTTIME,LOAD_DT,CTL_LOADING,COMMONSEGMENTOUID
0,7701412870,7c9a1b6b-d614-45d4-90e7-a8d4c6c0946e,1-CSOOQV8,SITEALLPRODAGG_01,SITE,RKO,None,None,None,0001-01-01 00:00:00,...,9999,9999,1.0,0,0,2021-02-19 11:06:19,2021-02-19 11:06:19,2021-03-01,6746703,None
1,7701412870,3c25f079-aea4-fc84-e054-90e2ba85d0c8,1-CSOOQV8,SITEALLPRODAGG_01,SITE,RKO,None,None,None,0001-01-01 00:00:00,...,9999,9999,1.0,0,0,2021-02-19 11:06:19,2021-02-19 11:06:19,2021-03-01,6746703,OzLrrbRQpjSB
2,7705653450,1c37cf13-b916-4812-a964-ecde1fec9a84,None,SITEALLPRODAGG_01,SITE,RKO,None,None,None,0001-01-01 00:00:00,...,9999,9999,1.0,0,0,2021-02-25 13:43:20,2021-02-25 13:43:20,2021-03-01,6749242,pvP5HgLBoy0A
3,7713754927,50ebd9cb-0812-4b88-86cd-9a1c7718c0c3,1-3E5E1NO,SITEALLPRODAGG_01,SITE,SALARYPROJ,None,None,None,0001-01-01 00:00:00,...,9999,9999,1.0,0,0,2021-02-24 16:13:20,2021-02-24 16:13:20,2021-03-01,6747224,None
4,771381015601,f035de5b-2b5a-43b6-9591-ee182c37007b,1-2EESX160,SITEALLPRODAGG_01,SITE,RKO,None,None,None,0001-01-01 00:00:00,...,9999,9999,1.0,0,0,2021-02-18 14:58:28,2021-02-18 14:58:28,2021-03-01,6744277,OfhHfmqZ5mH4


## Check saved DF via JDBC driver

In [26]:
print("\n,".join([col for col in df2.columns]))

INN
,SBBOLUSERID
,COMMONSEGMENTOUID
,ID_CRM
,SCENARIO_ID
,CHANNEL
,ID_PRODUCT
,MADEAL_PRODUCT_NORM
,MADEAL_PROD_TYPE_NAME
,MADEAL_APPL_CORE_TXT
,MADEAL_COMPLETE_DT
,HASPRODUCT
,ESSENSE
,RETURNCNT
,NUMBEROFSTEPS
,FUNNEL_RATE
,SUM_OFFER_PRIORITY
,SUM_OFFER_NONTOP
,MINSESSIONSTARTTIME
,MAXSESSIONSTARTTIME
,LOAD_DT
,CTL_LOADING


In [71]:
query = '''
(select /*+ parallel (12) */
    INN
    ,SBBOLUSERID
    ,COMMONSEGMENTOUID
    ,ID_CRM
    ,SCENARIO_ID
    ,CHANNEL
    ,ID_PRODUCT
    ,MADEAL_PRODUCT_NORM
    ,MADEAL_PROD_TYPE_NAME
    ,MADEAL_APPL_CORE_TXT
    ,CAST(MADEAL_COMPLETE_DT as VARCHAR2(40))
    ,HASPRODUCT
    ,ESSENSE
    ,RETURNCNT
    ,NUMBEROFSTEPS
    ,FUNNEL_RATE
    ,SUM_OFFER_PRIORITY
    ,SUM_OFFER_NONTOP
    ,MINSESSIONSTARTTIME
    ,MAXSESSIONSTARTTIME
    ,LOAD_DT
    ,CTL_LOADING       
FROM {}
where rownum < 1000)
'''.format(table_name)

sdf = sp.get_oracle(OracleDB('iskra4'), query)

In [72]:
sdf.show()

,INN,SBBOLUSERID,COMMONSEGMENTOUID,ID_CRM,SCENARIO_ID,CHANNEL,ID_PRODUCT,MADEAL_PRODUCT_NORM,MADEAL_PROD_TYPE_NAME,MADEAL_APPL_CORE_TXT,...,ESSENSE,RETURNCNT,NUMBEROFSTEPS,FUNNEL_RATE,SUM_OFFER_PRIORITY,SUM_OFFER_NONTOP,MINSESSIONSTARTTIME,MAXSESSIONSTARTTIME,LOAD_DT,CTL_LOADING
0,056103171674,3c25f094-8252-fc84-e054-90e2ba85d0c8,_KkecnZB8dfh,1-6KN84MN,SITEALLPRODAGG_01,SITE,RKO,None,None,None,...,SITE: РКО retargeting,9999,9999,9999,0,0,2020-05-19 15:10:16,2020-05-19 15:10:16,2020-05-22,3211373
1,056103171674,3c25f094-8252-fc84-e054-90e2ba85d0c8,_KkecnZB8dfh,1-6KN84MN,SITEALLPRODAGG_01,SITE,RKO,None,None,None,...,SITE: РКО retargeting,9999,9999,9999,0,0,2020-06-02 12:58:44,2020-06-03 10:57:45,2020-06-09,3355723
2,056103208042,3fe47f9c-e124-4cc4-b030-dbefa599a2c9,None,1-TA82PM9,CREDIT_01,SBBOL,CREDIT,None,None,None,...,SBBOL: CREDIT retargeting,3,5,1.00,0,0,2019-11-16 00:00:32,2020-01-26 21:40:20,2020-04-13,10241974
3,056103208042,3fe47f9c-e124-4cc4-b030-dbefa599a2c9,None,1-TA82PM9,STORIES_01,SBBOL,CREDIT,None,None,None,...,Retargeting SBBOL - stories interaction - Кред...,3,2,1.00,0,0,2019-11-16 00:00:32,2020-01-26 21:40:20,2020-04-19,10147356
4,0561032343,3c25f080-ca6c-fc84-e054-90e2ba85d0c8,"Hu8RRVdi4xSf,WoOmQPWWLtfu",1-2N933N0,STORIES_01,SBBOL,CREDIT,None,None,None,...,SBBOL: PRODUCT_NM retargeting,1,2,0.50,0,0,2020-06-01 12:47:04,2020-06-01 12:47:04,2020-06-08,3352371
5,056103272922,fa53e442-259e-4b66-9b38-ad0e31a555df,I_pPnJZg8npu,1-FC5E15R,CREDIT_01,SBBOL,CREDIT,None,None,None,...,SBBOL: Кредиты retargeting,0,5,0.40,0,0,2020-06-16 15:40:23,2020-06-16 15:40:23,2020-06-26,3551184
6,056103397569,cc829aa7-9695-4242-aeba-30c74361040f,J14sz2cP7sJt,1-ZV0FZL6,STORIES_01,SBBOL,BUSINESS_ANALYTICS,None,None,None,...,SBBOL: PRODUCT_NM retargeting,1,2,0.50,0,0,2020-05-08 14:04:32,2020-05-08 14:04:32,2020-05-12,3092685
7,056103543918,c29cae1f-125d-4681-8974-b3fde313fbc0,YF9-OzxYSIrK,1-1Q1J1EAJ,SITEALLPROD_01,SITE,RKO,None,None,None,...,SITE: РКО retargeting,9999,9999,9999,0,0,2019-10-16 09:14:43,2019-12-25 16:58:23,2020-04-13,10118034
8,056103548994,1abb7a43-f26d-4da9-aa91-81437946c9f9,None,1-STSINFE,STORIES_01,SBBOL,DEPOSIT,None,None,None,...,Retargeting SBBOL - stories interaction - Депо...,9,2,1.00,0,0,2020-01-08 16:31:08,2020-03-10 18:49:07,2020-04-19,10234916
9,056103548994,1abb7a43-f26d-4da9-aa91-81437946c9f9,8aQCH1Jun4op,1-STSINFE,STORIES_01,SBBOL,MYBOOK_ONLINE,None,None,None,...,Retargeting SBBOL - stories interaction - Моя ...,1,2,0.50,0,0,2019-10-12 15:18:15,2019-12-09 16:41:25,2020-04-19,10115381


## Extract via JDBS driver

In [ ]:
## !!! Don't forget to created an empty table before with CLOB types followed by all non-clob columns

In [129]:
table_name = 'GA_ALL_SCENARIOS_HIST'
sdf = hive.sql('''select * from {}.{} where (load_dt >= timestamp('2021-03-01') ) and 
                                            (load_dt <= timestamp('2021-03-06') )'''.format(conn_schema,table_name))

In [130]:
sdf = sdf.select([col for col in sdf.columns if col not in ("commonSegmentoUID")]+["commonSegmentoUID"])

In [131]:
sdf = sdf.select([col.upper() for col in sdf.columns])

typesmap={}
for column_name, column in sdf.dtypes:
    if column == 'string':
        if 'INN' in column_name.upper() or 'KPP' in column_name.upper():
            typesmap[column_name] = 'VARCHAR(20)'
        elif 'commonSegmentoUID'.upper() in column_name.upper():
            typesmap[column_name] = 'CLOB'
        else:
            typesmap[column_name] = 'VARCHAR(900)'
    elif column == 'int':
        typesmap[column_name] = 'INTEGER'
    elif column == 'bigint':
        typesmap[column_name] = 'LONG'
    elif column == 'timestamp':
        typesmap[column_name] = 'TIMESTAMP'
    elif column == 'float' or column == 'double':
        typesmap[column_name] = 'FLOAT'
    else:
        None
        
cols = ', '.join([col + ' ' + typesmap[col] for col in sdf.columns])
cols

'INN VARCHAR(20), SBBOLUSERID VARCHAR(900), ID_CRM VARCHAR(900), SCENARIO_ID VARCHAR(900), CHANNEL VARCHAR(900), ID_PRODUCT VARCHAR(900), MADEAL_PRODUCT_NORM VARCHAR(900), MADEAL_PROD_TYPE_NAME VARCHAR(900), MADEAL_APPL_CORE_TXT VARCHAR(900), MADEAL_COMPLETE_DT TIMESTAMP, HASPRODUCT INTEGER, ESSENSE VARCHAR(900), RETURNCNT INTEGER, NUMBEROFSTEPS INTEGER, FUNNEL_RATE VARCHAR(900), SUM_OFFER_PRIORITY INTEGER, SUM_OFFER_NONTOP INTEGER, MINSESSIONSTARTTIME TIMESTAMP, MAXSESSIONSTARTTIME TIMESTAMP, LOAD_DT TIMESTAMP, CTL_LOADING LONG, COMMONSEGMENTOUID CLOB'

In [132]:
table_name = "tmp_GA_ALL_SCENARIOS_HIST"

db = OracleDB('iskra4')
mode = 'append'
sdf \
    .write \
    .format('jdbc') \
    .mode(mode) \
    .option('url', 'jdbc:oracle:thin:@//'+db.dsn) \
    .option('user', db.user) \
    .option('password', db.password) \
    .option('dbtable', table_name) \
    .option('createTableColumnTypes', cols)\
    .option('driver', 'oracle.jdbc.driver.OracleDriver') \
    .save()

## Check saved DF

In [133]:
query = '''
(select /*+ parallel (12) */
    *      
FROM {}
where rownum < 1000)
'''.format(table_name)

sdf = sp.get_oracle(OracleDB('iskra4'), query)

In [134]:
sdf.show()

,INN,SBBOLUSERID,ID_CRM,SCENARIO_ID,CHANNEL,ID_PRODUCT,MADEAL_PRODUCT_NORM,MADEAL_PROD_TYPE_NAME,MADEAL_APPL_CORE_TXT,MADEAL_COMPLETE_DT,...,RETURNCNT,NUMBEROFSTEPS,FUNNEL_RATE,SUM_OFFER_PRIORITY,SUM_OFFER_NONTOP,MINSESSIONSTARTTIME,MAXSESSIONSTARTTIME,LOAD_DT,CTL_LOADING,COMMONSEGMENTOUID
0,1101054766,a1cb7fc8-e18a-429e-bd00-3554373369a0,None,WTA_SITE_NBS,SITE,OFD_PLATFORMA,None,None,None,None,...,9999,4,1.0,0,0,2021-02-24 13:59:47,2021-02-24 13:59:47,2021-03-01,6748040,2mSEKJj91PIp
1,1101054766,None,1-98CLV8X,WTA_SITE_NBS,SITE,OFD_PLATFORMA,None,None,None,None,...,9999,4,1.0,0,0,2021-02-24 13:59:47,2021-02-24 13:59:47,2021-03-01,6748040,2mSEKJj91PIp
2,7709528450,None,None,WTA_SITE_NBS,SITE,CNTERPRTY_VERIF,None,None,None,None,...,9999,4,1.0,0,0,2021-02-19 09:41:44,2021-02-19 09:41:44,2021-03-01,6746075,WNelv-IGcYTr
3,1327003570,80497cbd-37ec-402c-865f-293ed0531beb,1-189E0IG,DEPOSIT_01,SBBOL,DEPOSIT,None,None,None,None,...,2,3,0.33,0,0,2021-02-25 21:54:18,2021-02-25 21:54:18,2021-03-01,6760981,uKe4nXSJDGYq
4,1660216713,3c25f0ba-4130-1d5b-e054-90e2ba9a7f6c,1-2U6C4E3,DEPOSIT_01,SBBOL,DEPOSIT,None,None,None,None,...,1,3,0.33,0,0,2021-02-25 13:09:03,2021-02-25 13:09:03,2021-03-01,6760300,"T94aAmcgJ1Gx,kg2Bv7NFtiQ0,w3irrMVQfUtQ,9aT3kZc..."
5,2371000890,14dcab8d-c070-4e0a-ba12-fa48fc82606b,1-1O40XNB,DEPOSIT_01,SBBOL,DEPOSIT,None,None,None,None,...,1,3,0.33,0,1,2021-02-25 15:51:32,2021-02-25 15:51:32,2021-03-01,6749242,616w0_gMeIUe
6,244400865060,3c25f0ad-aa49-1d5b-e054-90e2ba9a7f6c,1-2DY5T4M,DEPOSIT_01,SBBOL,DEPOSIT,None,None,None,None,...,2,3,0.33,0,0,2021-02-24 05:34:30,2021-02-25 10:51:04,2021-03-01,6748040,"vUZ0B9cWypqR,LO8fqBXUdCQ9,pG3vvnTFoeqV"
7,3811469913,7d16ed60-25a9-498e-9a2b-da04b2a997e7,1-2GQUHPH7,DEPOSIT_01,SBBOL,DEPOSIT,None,None,None,None,...,2,3,1.00,0,0,2021-02-18 05:40:55,2021-02-18 05:40:55,2021-03-01,6744277,None
8,4201008991,3c25f0a5-4581-1d5b-e054-90e2ba9a7f6c,None,DEPOSIT_01,SBBOL,DEPOSIT,None,None,None,None,...,4,3,0.33,0,0,2021-02-24 11:30:58,2021-02-24 11:30:58,2021-03-01,6748040,eqf1d_zXDaeO
9,4715028202,3c25f0be-5bef-1d5b-e054-90e2ba9a7f6c,1-3DMAIZR,DEPOSIT_01,SBBOL,DEPOSIT,None,None,None,None,...,1,3,0.33,0,0,2021-02-18 15:56:05,2021-02-18 15:56:05,2021-03-01,6744277,"OYYv-RDQv4-w,d1mJ1uX2hFD8,r5X_G1ydLp6Y,pdxau9J..."
